<a href="https://colab.research.google.com/github/suvigyajain0101/NLP/blob/main/Intro_to_NLP_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# But, what is NLP?

NLP is a branch of AI, that deals with the interaction between computers and humans using the natural language. Most NLP techniques rely on Machine Learning to make sense of natural language. 
The ultimate objective of NLP is to read, decipher, understand and make sense of human languages

Typical NLP use cases - 
* Voice assistants, Ok Google, Siri, Cortana
* Language Translation, Google translate, Bing
* Gmail's spam detection filter
* MS Word's grammatical mistake feature

There are 2 broad approaches of handling NLP problems - Syntactic and Semantic Analysis

1. Syntax - Syntax in a code means the rules that need to followed for that particular programming language. Syntax in a language means the grammatical rules that need to be followed.
      Like POS tagging, lemmatization, stemming etc

2. Semantic - Semantic analytics dive deeper into the language to understand meaning that is conveyed by the text and sentence structure.
      Like NERs, NLU etc


Above is a short summary of this [Article](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32)

From machine learning perspective, NLP is about being as creative as we can get with converting our text to numbers. Why is that? Because ML models don't take text as input. In this notebook, we will use the following approach to break an NLP problem and derive insights from it - 
1. Get the text
2. Convert it into numbers
3. Modelling
4. Predictions


We will start with a baseline TF-IDF model and build on it using DNN, LSTM, GRU, Conv and Transfer learning.

In [1]:
# Import libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras import layers
import datetime
import numpy as np

# Let's define Universal Random State
random_state = 42

# For TensorBoard, let's define log storage directory
SAVE_DIR = 'model_logs'

#### The below code block can be used to download data from kaggle directly into Colab

In [2]:
## Data - We will use Kaggle's disaster tweets dataset

# Natural Language Processing with Disaster Tweets

In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 3.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73052 sha256=5a734a17da566577fd9e2f82d3ac979b08009cd53ccc1e568626b895e3476284
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"suvigyajain","key":"82b0c72e48196f9a45aa86528a87c74d"}'}

In [5]:
! mkdir ~/.kaggle


In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle competitions download -c 'nlp-getting-started'

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 95.3MB/s]


In [9]:
! mkdir input_data

In [10]:
! unzip nlp-getting-started.zip -d input_data

Archive:  nlp-getting-started.zip
  inflating: input_data/sample_submission.csv  
  inflating: input_data/test.csv     
  inflating: input_data/train.csv    


### Let's deep dive in the data

Kaggle created 3 files for us - train, test and sample-submission
We will read the files, do some EDA and then move forward

In [11]:
train_df = pd.read_csv('input_data/train.csv')
test_df = pd.read_csv('input_data/test.csv')

print("Train Data Size : ", len(train_df))
print("Test Data Size : ", len(test_df))

Train Data Size :  7613
Test Data Size :  3263


In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
# Data desc on Kaggle mentions that the location has this distribution - 
                  # USA -> 1%
                  # RoW -> 65%
                  # Null-> 33%

In [15]:
# What about keywords?
train_df[train_df['keyword'].notnull()].head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [16]:
# Are key words always part of the text? If so, then can we just ignore those?
train_df[train_df['keyword'].notnull()].apply(lambda x: x.keyword in x.text, axis=1)

31       True
32      False
33       True
34       True
35      False
        ...  
7578     True
7579     True
7580     True
7581     True
7582     True
Length: 7552, dtype: bool

In [17]:
# Not always true!! Let's see in the later stages if we can somehow use this added information

In [18]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

Another Binary Classification Problem. Fairly balanced target (60-40)

  1 - Disaster Related Tweet
  
  2 - Not related to Disaster

In [19]:
# Let's print out few 1s and 0s

positive_tweet_sample = train_df[train_df['target'] == 1]['text'].head().tolist()
negative_tweet_sample = train_df[train_df['target'] == 0]['text'].head().tolist()


print("Disaster Related Tweets : ", '\n')
for i in range(5):
  print('\t', positive_tweet_sample[i])
  print('\t', '-'*20)
print('\n')
print("Non-Disaster Related Tweets : ", '\n')
for i in range(5):
  print('\t', negative_tweet_sample[i])
  print('\t', '-'*20)

Disaster Related Tweets :  

	 Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
	 --------------------
	 Forest fire near La Ronge Sask. Canada
	 --------------------
	 All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
	 --------------------
	 13,000 people receive #wildfires evacuation orders in California 
	 --------------------
	 Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
	 --------------------


Non-Disaster Related Tweets :  

	 What's up man?
	 --------------------
	 I love fruits
	 --------------------
	 Summer is lovely
	 --------------------
	 My car is so fast
	 --------------------
	 What a goooooooaaaaaal!!!!!!
	 --------------------


Train-Test Split (Or Train Validation Split)

Since the testing data doesn't have any labels, we will have to split our training data into train and validation (or test). How about 90-10 for starters

In [20]:
# Define test_size
test_size = 0.1

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'].to_numpy(), train_df['target'].to_numpy(),
                                                                            test_size = test_size,
                                                                            random_state = random_state)

In [21]:
# Size of training and validation sets
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Convert text data into numbers

The next step would be to convert string data into numbers. Our labels are alreay numerical, so no need for LabelEncoder, but for tweet text, we surely need to convert.

There are 2 major approaches of converting text into numbers - 

* **Tokenization** - A mapping, or a lookup, for a word/sub-word/character. We can do following types of tokenization - 

      1. Word-Level Tokenization : Assign every word in the sequence it's own token. For example, in the sentence, 'I Love Pizza', we can assign I:0, Love:2, Pizza:3
      2. Character-Level Tokenization : Assign token to every character in the corpus. 
      3. Sub-word-Level Tokenization : This involves breaking words into parts and assigning tokens to each part. 
                    For example, 'I love Pineapple', can be broken
                    into 'I Lo ve pine app le' and then assign 
                    tokens to these sub-words. In case of sub-word
                    tokenization, a word might have multiple tokens

* **Embeddings** - Embedding is a representation of natural language which can be learned. This representation comes in the form of a Feature vector. For example, the word 'Football' can be represented by a 5-d feature vector : [-0.3456, 0.2352, 0.3454, 0.2576, 0.9865]. Importantly, the size of the feature vector is tune-able. We can use embeddings in 2 ways : 

      1. Create your own embedding : Once your text has been converted to
                      numbers (required for Embedding), you can pass this
                      to Keras Embedding Layer and an embedding
                      representation will be learned during model training
      2. Use a pre-learned embedding : `Transfer Learning`. 
                      The power of Deep Learning. You can use pre-created
                      embedding layers and fine-tune them on your
                      own purpose. The benefit here is, for example BERT
                      is trained on entire wikipedia. It is not possible
                      that every time we perform a sentiment analysis we
                      train it for months on such a huge corpus. We rather
                      use the results from the previous training and
                      adapt it to our purpose.


To tokenize our words, we'll use the helpful preprocessing layer `tf.keras.layers.experimental.preprocessing.TextVectorization`.

The TextVectorization layer takes the following parameters:

* max_tokens - The maximum number of words in your vocabulary (e.g. 20000 or the number of unique words in your text), includes a value for OOV (out of vocabulary) tokens.
* standardize - Method for standardizing text. Default is "lower_and_strip_punctuation" which lowers text and removes all punctuation marks.
* split - How to split text, default is "whitespace" which splits on spaces.
* ngrams - How many words to contain per token split, for example, ngrams=2 splits tokens into continuous sequences of 2.
* output_mode - How to output tokens, can be "int" (integer mapping), "binary" (one-hot encoding), "count" or "tf-idf". See documentation for more.
* output_sequence_length - Length of tokenized sequence to output. For example, if output_sequence_length=150, all tokenized sequences will be 150 tokens long.
* pad_to_max_tokens - If True (default), the output feature axis will be padded to max_tokens even if the number of unique tokens in the vocabulary is less than max_tokens.



In [22]:
# What's the number of distinct words in the corpus?
from collections import Counter
# Instantiate Counter
results = Counter()
# Convert text to lower case, split on whitespace, and create dict
train_df['text'].str.lower().str.split().apply(results.update)
# Shape of Dictionary
print(len(results.keys()))

27983


In [23]:
# Close to 30K if we include test data as well. We'll choose 10K as the vocab_size (or max_tokens)

In [24]:
# Average tweet length?
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [25]:
# We will choose output_sequence_length as 15, since Average Tweet Length is 15

In [26]:
# Initialize Text Vectorizer
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vocab_size = 10000
output_len = 15

text_vectorizer = TextVectorization(max_tokens=vocab_size,
                                    output_sequence_length = output_len,
                                    output_mode = 'int')

# Adapt the vectorizer created above to our training data (Adapt is like fit-transform)
text_vectorizer.adapt(train_sentences)

In [27]:
# Let's see how this vectorizer is working on few sample text

sample_corpus = [['I love Pizza'],
                 ['I love Football'],
                 ['The dog loves cricket']]
text_vectorizer(sample_corpus)

<tf.Tensor: shape=(3, 15), dtype=int64, numpy=
array([[   8,  107, 3526,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   8,  107, 1528,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   2, 1014, 2401, 3964,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [28]:
# So, every output is of length 15. Same word gets same token
# Also, notice that love and loves get different tokens. Ideally we should convert the word into it's stem and assign tokens to the stem

In [29]:
# Explore the vectorizer a bit more

# All words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary()
# First 5 words in the vocab
first5_words = words_in_vocab[:5]
# Last 5 words in the vocab
last5_words = words_in_vocab[-5:]

print("Total numbers of words in the vocab : ", len(words_in_vocab))
print("Top 5 most popular words in the vocab : ", first5_words)
print("Top 5 least popular words in the vocab : ", last5_words)

Total numbers of words in the vocab :  10000
Top 5 most popular words in the vocab :  ['', '[UNK]', 'the', 'a', 'in']
Top 5 least popular words in the vocab :  ['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


### Create an embedding using Keras Embedding Layer

Next, we convert the tokens to an embedding. One major advantage is that an embedding layer can be train-able, so we can update the tokens if we've obtained from text vectorizer

Main parameters we are looking for - 
1. input_dim = Size of the vocabulary, i.e. maximum integer index + 1.
2. input_length = Length of input sequences being passed to the embedding (15 here)
3. output_dim = Output array size (Map input to this dim. If 100, we get output of size (m, input_len, 100) where m is the number of training examples)

Please have a look at the official doc for better understanding - [EmbeddingLayer](https://keras.io/api/layers/core_layers/embedding/)

In [30]:
from tensorflow.keras import layers

embedding_output = 128

embedding = layers.Embedding(input_dim = vocab_size,
                             output_dim = embedding_output,
                             input_length = output_len)

In [31]:
# Let's see what exactly this 'thing' is doing
random_sentence = random.choice(train_sentences)
print("Original Text : ", random_sentence)

# Let's embed this random sentence
print("\n\nEmbedded version")
embed_random_sentence = embedding(text_vectorizer([random_sentence]))
embed_random_sentence

Original Text :  I BET YOU DIDNT KNOW I KICK BOX TOO! https://t.co/rBrw8pWiPJ


Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01894781, -0.00726347, -0.04992763, ..., -0.0409349 ,
         -0.03139122,  0.00509682],
        [-0.04105787, -0.02647586,  0.03633232, ..., -0.04589159,
         -0.02794155,  0.04608059],
        [-0.0431283 , -0.00957646, -0.01949136, ..., -0.03312575,
          0.00886799, -0.04914499],
        ...,
        [-0.02573162, -0.01475918, -0.00649576, ..., -0.00397563,
          0.03644766, -0.04618732],
        [-0.02573162, -0.01475918, -0.00649576, ..., -0.00397563,
          0.03644766, -0.04618732],
        [-0.02573162, -0.01475918, -0.00649576, ..., -0.00397563,
          0.03644766, -0.04618732]]], dtype=float32)>

In [32]:
# Take closer look at tensor shape - <tf.Tensor: shape=(1, 15, 128), dtype=float32
# 1 input example. 15 words in every tweet. Or 15 max/min tokens per tweet. 128 dimensional embedding. So, every token gets converted to 128 dims


# Check out a single token's embedding
embed_random_sentence[0][0], len(embed_random_sentence[0][0])

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 1.89478137e-02, -7.26347044e-03, -4.99276295e-02,  2.74675004e-02,
        -4.64537144e-02, -1.78060159e-02,  3.01951654e-02, -1.20282061e-02,
         3.25137414e-02, -2.22574472e-02, -2.61754990e-02,  2.43568160e-02,
        -3.20677981e-02, -2.04050783e-02,  1.11190900e-02,  3.04908417e-02,
         2.73795463e-02,  4.15088199e-02,  4.29514088e-02, -4.59040664e-02,
         8.98696110e-03, -4.43557277e-02,  3.31128575e-02, -1.30199306e-02,
        -1.35957375e-02, -1.53317079e-02, -3.20808515e-02,  4.22249548e-02,
         2.70626061e-02, -2.92564519e-02,  3.07726301e-02,  3.34253199e-02,
         2.40153186e-02,  2.63348483e-02, -1.46042705e-02, -1.15582831e-02,
         3.43162306e-02, -2.30682846e-02, -2.15090513e-02, -1.61741488e-02,
        -2.90071126e-02, -4.85444330e-02, -1.82237998e-02, -1.39297470e-02,
         1.47400834e-02,  4.77270372e-02, -3.42938453e-02, -3.70769128e-02,
         3.79866101e-02, -4.37525511e-0

In [33]:
# So, every token (or a word if we use word-level tokenization) gets mapped into a 128 dimensional space

# What a beautiful matrix. LOL!!

## Modelling

We'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data

In [34]:
# Since we will doing a lot of experimentation, it's a good idea we create some base functions
# Following function evaluates: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

def compare_baseline_to_new_results(baseline_results, new_model_results):
  """
  Function to compare results between baseline model and any other model
  Both baseline and new model's results are dictionaries of Accuracy, F1, Precision, Recall
  """
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

### Model 0 - Baseline NB Model

Just like TF, Scikit Learn models also don't take strings as inputs (DUHH!!)

So, we will use [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) score for each word to convert text to number

Also, we will create Sklearn pipeline to get the results

But before that, a bit about TF-IDF. TF-IDF evaluated how relevant a word is in a collection of documents. It has 2 terms - 

1. Term Frequency (TF) - This measures the frequency of a word in a doc. We normalise the effect of the words by the total number of words in the doc.

> TF = Freq of word W in Doc D / Total number of words in D

2. Inverse Document Frequency (IDF) - This measures the importance of a document in the corpus. We are only interested in the number of docs the word W is present and NOT IN THE FREQUENCY.

> IDF Definition = Inverse of (Number of docs in which word W occurs / Total number of Docs in Corpus). However, if number of docs in which W occurs is zero, then we might get DivisionByZeroError
    
So, IDF is defined as - 

> IDF = log(N / (df + 1)), where df is the number of docs in which W occurs

So, putting it all together, TF-IDF score for a word W is -
  
    TFIDF = tf(w, d) * log(N / (df + 1)
            where w : word for which TFIDF is being calculated
                  d : current document
                 df : number of docs in which w occurs
                  N : Total number of docs


Notice that TF-IDF normalizes the score for words like in, the, as, so (Stopwords). TF score will be high, but since these words will be present in most of the docs, IDF score will be very low. Still, it's a good practice to perform Data Cleaning


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and MNB pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

# Make predictions
baseline_preds = model_0.predict(val_sentences)
print("Few predictions : ", baseline_preds[:10])

# Evaluate the model on Testing data
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

# Evaluation metrics
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

Few predictions :  [0 0 0 0 0 0 0 0 0 1]
Our baseline model achieves an accuracy of: 77.82%


{'accuracy': 77.82152230971128,
 'f1': 0.7703527809038113,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129}

### Model 1 - A simple Dense Model

Too simple I'd say. Basically we are taking text and labels as input, do the tokenization, create embedding, convert embedding into lower dimension (like avg) and then pass it to 1 fully connected layer

In [36]:
# We have 1-d input. Tweet as a string. 
inputs = layers.Input(shape=(1,), dtype = 'string')
# Tokenize the text
x = text_vectorizer(inputs)
# Create embeddings using embedding layer created above
x = embedding(x)
# Convert to lower dimesnion, using Average
x = layers.GlobalAveragePooling1D()(x)

# Create outputs - since the output is Binary, we'll use Sigmoid
outputs = layers.Dense(1, activation = 'sigmoid')(x)

# Create the model
model_1 = tf.keras.Model(inputs, outputs, name = 'model_1_dense')

# Compile the model
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get a short summary of the model
model_1.summary()

# Fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = 'Simple_Dense_Model')])

# Evaluate the model
model_1.evaluate(val_sentences, val_labels)

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________
Saving TensorBoard log files to: model_logs/Simple_Dense_Model/20210722-183851
Epoch 1/5
215/215 [===========

[0.5006482601165771, 0.7795275449752808]

In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.7795


[0.5006482601165771, 0.7795275449752808]

In [38]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

array([[0.0302527 ],
       [0.14803392],
       [0.35578954],
       [0.02387226],
       [0.6122947 ],
       [0.13952953],
       [0.0191409 ],
       [0.26589465],
       [0.10997969],
       [0.89926887]], dtype=float32)

Sigmoid function returns probabilities and these probabilities need to be converted to 1,0.

We will use a simple round function to convert probabilities to binary. This means we use 0.5 as a cut-off. However, for experimentation, we can ROC curve to determine threshold

In [39]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)>

In [40]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 77.95275590551181,
 'f1': 0.7763047940993759,
 'precision': 0.781906099911785,
 'recall': 0.7795275590551181}

In [41]:
# Is our simple Keras model better than our baseline model?

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

In [42]:
# Compare Model's results to Baseline model

compare_baseline_to_new_results(baseline_results=baseline_results, 
                                new_model_results=model_1_results)

Baseline accuracy: 77.82, New accuracy: 77.95, Difference: 0.13
Baseline precision: 0.79, New precision: 0.78, Difference: -0.01
Baseline recall: 0.78, New recall: 0.78, Difference: 0.00
Baseline f1: 0.77, New f1: 0.78, Difference: 0.01


### RNN

Next, we look at another type of ANN architecture - Recurrent Neural Networks. These are based on simple premise of using the information from previous state and define next steps using inputs from the previous state. This is specially useful in problems like Text, Audio etc (basically Sequences). 

Recurrent neural networks can be used for a number of sequence-based problems:

1. **One to one**: one input, one output, such as image classification.
2. **One to many**: one input, many outputs, such as image captioning (image input, a sequence of text as caption output).
3. **Many to one**: many inputs, one outputs, such as text classification (classifying a Tweet as real diaster or not real diaster).
4. **Many to many**: many inputs, many outputs, such as machine translation (translating English to Spanish) or speech to text (audio wave as input, text as output).

RNNs can be broadly seen under following variants - 

1. Long Short Term Memory (LSTM) Units
2. Gated Recurrent Units (GRU)
3. Bi-Directional RNNs



### Model 2 - LSTM

LSTM is a type of RNN based on the paradigm of using information from previous state. Long Short Term Memory networks allow the information to persist, unlike traditional neural nets, and can be thought of as multiple copies of the same network. Same can be applied to RNNs. However, as the length of the state grows, i.e., the distance in the past we have to go, RNN become slow. This problem is overcome by LSTMs. LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!


The model structure remains pretty same as Model 1, just that we will add an LSTM layer between embedding and output - 

`Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)`


In [45]:
from tensorflow.keras import layers

# Define the input layer
inputs = layers.Input(shape=(1,) , dtype = "string")

# Vectorize the inputs
x = text_vectorizer(inputs)

# Convert to embeddings
x = embedding(x)

# Add LSTM layer
x = layers.LSTM(64)(x)

# Define output
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model_2 = tf.keras.Model(inputs, outputs, name = 'Model_2_LSTM')

In [46]:
# Compile the model
model_2.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [47]:
# Model Summary
model_2.summary()

Model: "Model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Fit the model

model_2.fit(train_sentences,
            train_labels,
            epochs = 5,
            validation_data = (val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                    "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20210722-185940
Epoch 1/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0718 - accuracy: 0.9707 - val_loss: 0.9110 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 7s 35ms/step - loss: 0.0602 - accuracy: 0.9736 - val_loss: 1.3271 - val_accuracy: 0.7467
Epoch 3/5
215/215 [==============================] - 7s 34ms/step - loss: 0.0518 - accuracy: 0.9762 - val_loss: 1.3306 - val_accuracy: 0.7388
Epoch 4/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0432 - accuracy: 0.9796 - val_loss: 1.2830 - val_accuracy: 0.7402
Epoch 5/5
215/215 [==============================] - 8s 35ms/step - loss: 0.0448 - accuracy: 0.9774 - val_loss: 1.4025 - val_accuracy: 0.7388


In [50]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

((762, 1), array([[7.1352720e-04],
        [1.6766280e-02],
        [2.7158856e-04],
        [9.9003060e-05],
        [9.9286509e-01],
        [6.8951160e-02],
        [9.7978693e-05],
        [5.7793856e-03],
        [2.2467971e-03],
        [9.9994361e-01]], dtype=float32))

In [51]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)>

In [52]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 73.88451443569554,
 'f1': 0.7376499139456566,
 'precision': 0.7378208860176874,
 'recall': 0.7388451443569554}

In [53]:
# Compare model 2 to baseline
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 77.82, New accuracy: 73.88, Difference: -3.94
Baseline precision: 0.79, New precision: 0.74, Difference: -0.06
Baseline recall: 0.78, New recall: 0.74, Difference: -0.04
Baseline f1: 0.77, New f1: 0.74, Difference: -0.03


### Model 3 - GRU

Gated Recurrent Units are similar to LSTM, just have lesser parameters. We will use the same process as LSTMs, where instead of LSTM layer, we will use GRU layer - 

`Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)`

In [63]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

# Define the input layer
inputs = layers.Input(shape=(1,) , dtype = "string")

# Vectorize the inputs
x = text_vectorizer(inputs)

# Convert to embeddings
x = embedding(x)

# Add LSTM layer
x = layers.GRU(64)(x)

# Define output
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [55]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [56]:
# Get a summary of the GRU model
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20210722-190641
Epoch 1/5
215/215 [==============================] - 10s 37ms/step - loss: 0.1353 - accuracy: 0.9479 - val_loss: 0.9215 - val_accuracy: 0.7349
Epoch 2/5
215/215 [==============================] - 7s 33ms/step - loss: 0.0682 - accuracy: 0.9747 - val_loss: 0.8619 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 7s 33ms/step - loss: 0.0535 - accuracy: 0.9766 - val_loss: 1.1552 - val_accuracy: 0.7205
Epoch 4/5
215/215 [==============================] - 7s 34ms/step - loss: 0.0435 - accuracy: 0.9788 - val_loss: 1.4458 - val_accuracy: 0.7480
Epoch 5/5
215/215 [==============================] - 7s 33ms/step - loss: 0.0399 - accuracy: 0.9793 - val_loss: 1.5678 - val_accuracy: 0.7467


In [58]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1), array([[1.8187463e-03],
        [7.9035759e-04],
        [1.3765693e-04],
        [3.3465276e-05],
        [9.9695539e-01],
        [3.9073646e-02],
        [8.3327839e-05],
        [2.7772188e-03],
        [6.3627958e-04],
        [9.9987745e-01]], dtype=float32))

In [59]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)>

In [60]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 74.67191601049869,
 'f1': 0.7451761796828422,
 'precision': 0.7458383161889279,
 'recall': 0.7467191601049868}

In [61]:
# Compare to baseline
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 77.82, New accuracy: 74.67, Difference: -3.15
Baseline precision: 0.79, New precision: 0.75, Difference: -0.05
Baseline recall: 0.78, New recall: 0.75, Difference: -0.03
Baseline f1: 0.77, New f1: 0.75, Difference: -0.03


### Model 4 - Bi-Directional RNNs

We have already trained 2 variants of RNNs - LSTM and GRU. Next, we look at another type of RNN - Bi-Directional RNN. 

A standard RNN will process the data from left to right. Bi-Directional RNN will process the sequence from left-right and then also from right-left. Intuitively, someone reads a sentence from left to right and if it doesn't make much sense, go over the sentence again from right to left. 

More often than often, Bi-Directional RNNs provide improved performance compared to vanilla RNNs. However, this comes at the cost of longer training times and increased model parameters, since left-right and right-left.

Here again, we will follow the same structure as in model 3 and 4 - 

`Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)`

In [64]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

# Define the input layer
inputs = layers.Input(shape=(1,) , dtype = "string")

# Vectorize the inputs
x = text_vectorizer(inputs)

# Convert to embeddings
x = embedding(x)

# Add Bi-Directional layer - Notice how we wrapped LSTM layer inside TF Bidirectional. This can be done with any RNN layer
x = layers.Bidirectional(layers.LSTM(64))(x)

# Define output
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [65]:
# Compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [66]:
# Get a summary of our bidirectional model
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


A lot more parameters!!

In [68]:
# Fit the model (takes longer because of the bidirectional layers)
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20210722-192019
Epoch 1/5
215/215 [==============================] - 15s 52ms/step - loss: 0.0986 - accuracy: 0.9695 - val_loss: 1.0805 - val_accuracy: 0.7323
Epoch 2/5
215/215 [==============================] - 10s 44ms/step - loss: 0.0411 - accuracy: 0.9809 - val_loss: 1.5130 - val_accuracy: 0.7402
Epoch 3/5
215/215 [==============================] - 10s 46ms/step - loss: 0.0377 - accuracy: 0.9804 - val_loss: 1.3919 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0390 - accuracy: 0.9812 - val_loss: 1.5921 - val_accuracy: 0.7362
Epoch 5/5
215/215 [==============================] - 10s 44ms/step - loss: 0.0397 - accuracy: 0.9799 - val_loss: 1.3411 - val_accuracy: 0.7336


In [69]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[6.1818957e-04],
       [2.4544120e-02],
       [6.1151385e-04],
       [8.2206927e-05],
       [9.8854160e-01],
       [4.6894762e-01],
       [5.4006330e-05],
       [9.9662095e-02],
       [1.8688738e-03],
       [9.9977863e-01]], dtype=float32)

In [70]:
# Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)>

In [71]:
# Calculate bidirectional RNN model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 73.35958005249343,
 'f1': 0.7335534449181739,
 'precision': 0.7335136264316362,
 'recall': 0.7335958005249343}

In [72]:
# Check to see how the bidirectional model performs against the baseline
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 77.82, New accuracy: 73.36, Difference: -4.46
Baseline precision: 0.79, New precision: 0.73, Difference: -0.06
Baseline recall: 0.78, New recall: 0.73, Difference: -0.04
Baseline f1: 0.77, New f1: 0.73, Difference: -0.04
